# VacationPy
----



In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
city_data = pd.read_csv("../output_data/clean_city_data.csv")

# delete extra index Unnamed: 0
del city_data['Unnamed: 0']

city_data

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
0,Saint-Philippe,RE,-21.36,55.77,56.79,82,75,4.70
1,Flin Flon,CA,54.77,-101.86,75.20,50,40,8.05
2,Bredasdorp,ZA,-34.53,20.04,57.20,87,0,8.05
3,Lebu,CL,-37.62,-73.65,53.31,78,100,11.54
4,New Norfolk,AU,-42.78,147.06,43.29,85,100,1.99
...,...,...,...,...,...,...,...,...
541,Mugumu,TZ,-1.85,34.70,66.56,65,91,2.55
542,Atikokan,CA,48.76,-91.62,56.84,93,100,6.22
543,Busembatia,UG,0.77,33.61,67.77,82,95,2.10
544,Rock Sound,BS,24.90,-76.20,82.98,79,97,14.18


### Humidity Heatmap
* Configure gmaps.

In [13]:
gmaps.configure(api_key=g_key)

* Use the Lat and Lng as locations and Humidity as the weight.

In [14]:
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity"]

In [15]:
# Customize the size of the figure
figure_layout = {
    'width': '989px',
    'height': '388px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(46.0, -5.0), zoom_level=2)

* Add Heatmap layer to map.

In [16]:
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=np.max(weight), point_radius=3)

In [17]:
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='388px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
hotel_df = city_data.loc[(city_data["Wind Speed"] <= 15) & (city_data["Cloudiness"] <= 10) &(city_data["Temp"] >= 75) & (city_data["Temp"] <= 95) & (city_data["Humidity"] <= 70)].dropna()

hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
27,Post Falls,US,47.72,-116.95,88.48,32,1,8.05
49,Province of Mantua,IT,45.17,10.78,77.07,64,0,3.00
76,Albany,US,42.60,-73.97,85.39,41,4,8.41
118,Broome,US,42.25,-75.83,80.15,54,1,3.36
158,Dalbandin,PK,28.89,64.41,82.20,53,0,9.08
174,Craig,US,40.52,-107.55,82.13,20,1,9.17
193,Mitsamiouli,KM,-11.38,43.28,76.95,61,0,9.75
205,Bilma,NE,18.69,12.92,94.93,21,1,9.73
213,Kermanshah,IR,34.31,47.06,87.80,8,0,9.17
231,Plainview,US,40.78,-73.47,89.02,43,1,13.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.

In [19]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
27,Post Falls,US,47.72,-116.95,88.48,32,1,8.05,
49,Province of Mantua,IT,45.17,10.78,77.07,64,0,3.00,
76,Albany,US,42.60,-73.97,85.39,41,4,8.41,
118,Broome,US,42.25,-75.83,80.15,54,1,3.36,
158,Dalbandin,PK,28.89,64.41,82.20,53,0,9.08,
174,Craig,US,40.52,-107.55,82.13,20,1,9.17,
193,Mitsamiouli,KM,-11.38,43.28,76.95,61,0,9.75,
205,Bilma,NE,18.69,12.92,94.93,21,1,9.73,
213,Kermanshah,IR,34.31,47.06,87.80,8,0,9.17,
231,Plainview,US,40.78,-73.47,89.02,43,1,13.87,


* Set parameters to search for hotels with 5000 meters.

In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

* Hit the Google Places API for each city's coordinates.

In [21]:
print('Beginning Hotel Data Retrieval')
print('------------------------------')

for index, row in hotel_df.iterrows():
    # get city name location from hotel df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed

print('------------------------------')
print(f"End of City Data Retrieval for Hotels near our select cities")
print('------------------------------')

Beginning Hotel Data Retrieval
------------------------------
Retrieving Results for Index 27: Post Falls.
Closest hotel in Post Falls is Red Lion Hotel Templin's on the River.
------------
Retrieving Results for Index 49: Province of Mantua.
Closest hotel in Province of Mantua is Casa Museo Palazzo Valenti Gonzaga.
------------
Retrieving Results for Index 76: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 118: Broome.
Missing field/result... skipping.
------------
Retrieving Results for Index 158: Dalbandin.
Closest hotel in Dalbandin is Al-Dawood Hotel & Restaurant.
------------
Retrieving Results for Index 174: Craig.
Closest hotel in Craig is Quality Inn & Suites.
------------
Retrieving Results for Index 193: Mitsamiouli.
Closest hotel in Mitsamiouli is Moina Lisa.
------------
Retrieving Results for Index 205: Bilma.
Missing field/result... skipping.
------------
Retrieving Results for Index 213: Kermanshah.
Closest hotel in Kermanshah is Sin

* Store the first Hotel result into the DataFrame.

In [22]:
hotel_df

,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
27,Post Falls,US,47.72,-116.95,88.48,32,1,8.05,Red Lion Hotel Templin's on the River
49,Province of Mantua,IT,45.17,10.78,77.07,64,0,3.00,Casa Museo Palazzo Valenti Gonzaga
76,Albany,US,42.60,-73.97,85.39,41,4,8.41,
118,Broome,US,42.25,-75.83,80.15,54,1,3.36,
158,Dalbandin,PK,28.89,64.41,82.20,53,0,9.08,Al-Dawood Hotel & Restaurant
174,Craig,US,40.52,-107.55,82.13,20,1,9.17,Quality Inn & Suites
193,Mitsamiouli,KM,-11.38,43.28,76.95,61,0,9.75,Moina Lisa
205,Bilma,NE,18.69,12.92,94.93,21,1,9.73,
213,Kermanshah,IR,34.31,47.06,87.80,8,0,9.17,Sina Hotel
231,Plainview,US,40.78,-73.47,89.02,43,1,13.87,Marriott Melville Long Island


* Plot markers on top of the heatmap.

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# adding marker layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='388px', margin='0 auto 0 auto', padding='1px', wi…